In [ ]:
pip install tensorflow

# Data analysis

### Imports

In [ ]:
# Import list

import numpy as np
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelBinarizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

### Read Data 

In [ ]:
# Read from path 

PATH = "0. Projects/3/Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

# Read the text file
with open(PATH, 'r', encoding='utf-8') as file:
    text = file.read()

text = text.lower()
text

### Divide the set

In [ ]:
### Try with different separators in split

# for line in text.split('.' and '?' and '!' and ':' and ';'):
# for line in text.split'\n'):
# for line in text.split('.'):
#     print(tokenizer.texts_(to_sequences([line])[0])


### Try with all replace

# text = text.replace('\n', ' ').replace('!', '.').replace(':', '.').replace('+', ' ')
# text

In [ ]:
# Try with regular expressions

DIVIDERS_ORIGINAL = "\n"
DIVIDERS_ALL = "[,.!?:;\"]|\n\n|--| and | that | which "
DIVIDERS_MIN = "[.!]|\n\n"
DIVIDERS_BAL = "[,.!?]|\n\n|--"
divide_set = False

text_try = text

if divide_set:
    # Delete cover of book and extra information
    text_try = text[980:-550]

# Split following the dividers given
text_try = re.split(DIVIDERS_MIN, text_try)

# Delete all the new line comments 
text_try = [el.replace('\n', '') for el in text_try]

text_try

### Tokenization

In [ ]:
# Create Tokenizer object in python

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts([text])
# total_words = len(tokenizer.word_index) + 1

tokens = word_tokenize(text)
vocabulary = set(tokens)
total_words = len(vocabulary) + 1

word_to_idx = {word:idx for idx, word in enumerate(vocabulary)}

print(f"total_words: {total_words}")
print("Índice de palabras:", word_to_idx)

In [ ]:
# Create input-output sequences

input_sequences = []
# for line in text_try:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         n_gram_sequence = token_list[:i+1]
#         input_sequences.append(n_gram_sequence)

input_sequences = []
for line in text_try:
    line_list = line.rstrip(",.;:").split(' ')

    token_list = []
    for char in line_list:
        if char in word_to_idx.keys():
            token_list.append(word_to_idx[char])

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

In [ ]:
# Get the max value to add padding to other entries

average = 0
for seq in input_sequences:
    average += len(seq) 

max_sequence_len, value = max([(len(seq), seq) for seq in input_sequences])

# max_sequence_len = max([len(seq) for seq in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_seq_pad = np.array([np.pad(seq, (max_sequence_len - len(seq), 0), mode='constant') for seq in input_sequences])

print (f"average = {average / len(input_sequences)}")
input_seq_pad

In [ ]:
# Get index of the longest line

# token_list = []
# for line in text_try:
#     token_list.append(tokenizer.texts_to_sequences([line])[0])

# token_list.index(value)


# Print longest line

# for el in value:
#     for word, index in tokenizer.word_index.items():
#         if index == el:
#             print(word)
#             break
# len(input_sequences)
# input_sequences

### Separate input and output

In [ ]:
X = input_seq_pad[:, :-1]
y = input_seq_pad[:, -1]

X

In [ ]:
y